In [ ]:
import pandas as pd
label_df=pd.read_csv('/content/drive/MyDrive/data/train.csv')
label_df.head()

In [ ]:
import os
from glob import glob

def get_train_data(data_dir):
  img_path_list=[]
  label_list=[]

  # get image path
  img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
  img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))

  # get label
  label_list.extend(label_df['label'])

  return img_path_list,label_list

def get_test_data(data_dir):
  img_path_list=[]

  # get image path
  img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
  img_path_list.sort(key= lambda x:int(x.split('/')[-1].split('.')[0]))
  print(img_path_list)

  return img_path_list

In [ ]:
print(label_df.info())
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB
None
'''

In [ ]:
all_img_path,all_label=get_train_data('/content/drive/MyDrive/data/train')
test_img_path=get_test_data('/content/drive/MyDrive/data/test')

In [ ]:
print(all_label[:5])
print(all_img_path[:5])
print(test_img_path[:5])

In [ ]:
import torch
import torch.nn as nn

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#GPU 체크 및 할당
if torch.cuda.is_available():    
    #device = torch.device("cuda:0")
    print('Device:', device)
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [ ]:
#하이퍼 파라미터 튜닝

CFG = {
    'IMG_SIZE':128, #이미지 사이즈
    'EPOCHS':150, #에포크
    'LEARNING_RATE':2e-2, #학습률
    'BATCH_SIZE':32, #배치사이즈
    'SEED':41, #시드
}

In [ ]:
# Seed 고정
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import cv2

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,img_path_list,label_list,train_mode=True,transforms=None): #필요한 변수들을 선언
    self.transforms=transforms
    self.train_mode=train_mode
    self.img_path_list=img_path_list
    self.label_list=label_list

  def __getitem__(self,index):  # index 번쨰 data를 return
    img_path=self.img_path_list[index]
    image=cv2.imread(img_path)
    if self.transforms is not None:
      image=self.transforms(image)

    if self.train_mode:
      label=self.label_list[index]
      return image,label

    else:
      return image
    
  def __len__ (self):
    return len(self.img_path_list)

In [ ]:
tempdataset=CustomDataset(all_img_path,all_label,train_mode=False)
'''image = cv2.imread('/content/drive/MyDrive/data/train/001.PNG')
plt.imshow(image)'''

plt.imshow(tempdataset.__getitem__(88))

In [ ]:
print(tempdataset[20].shape)

In [ ]:
train_len=(int(len(all_img_path) * 0.75))
val_len=int(len(all_img_path) * 0.25)

train_img_path=all_img_path[:train_len]
train_label=all_label[:train_len]

vali_img_path=all_img_path[train_len:]
vali_label=all_label[train_len:]

In [ ]:
print(len(all_img_path))
print(len(test_img_path))
print(train_len)
print(val_len)

In [ ]:
train_transform=transforms.Compose([
    transforms.ToPILImage(),  # Numpy 배열에서 PIL 이미지로
    transforms.Resize([CFG['IMG_SIZE'],CFG['IMG_SIZE']]), # 이미지 사이즈 변형
    transforms.ToTensor(), # 이미지 데이터를 tensor
    transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5)), # 이미지 정규화
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip()
    ])

test_transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([CFG['IMG_SIZE'],CFG['IMG_SIZE']]),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))
])

In [ ]:
# Get Dataloader

# CustomDataset class를 통하여 train dataset생성
train_dataset=CustomDataset(train_img_path, train_label,train_mode=True,transforms=train_transform)
# 만든 train dataset를 DataLoader에 넣어 batch 만들기
train_loader=DataLoader(train_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=True,num_workers=0)

# validation도 적용
vali_dataset=CustomDataset(vali_img_path,vali_label,train_mode=True,transforms=test_transform)
vali_loader=DataLoader(vali_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=False,num_workers=0)

In [ ]:
train_batches=len(train_loader)
vali_batches=len(vali_loader)

print('total train imgs:',train_len,'/ total train bathcnes:', train_batches)
print('total valid imgs:',val_len,'/ total valid batches:', vali_batches)

In [ ]:
import torch.optim as optim
from tqdm.auto import tqdm
import torch.nn as nn
from torchvision import models

resnet=models.resnet50(pretrained=True).to(device)
for param in resnet.parameters():
  param.requires_grad=False

in_features=resnet.fc.in_features

classifier=nn.Sequential(
    nn.Linear(in_features,1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024,10),

)

resnet.fc=classifier


criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=resnet.parameters(),lr=CFG['LEARNING_RATE'])
scheduler=None

resnet.to(device)

In [ ]:
from torchsummary import summary
summary(resnet,(3,128,128))

In [ ]:
from tqdm import tqdm

def train(model,optimizer,train_loader,scheduler,device):
  model.to(device)
  n=len(train_loader)
  best_loss=10000

  for epoch in range(1,CFG['EPOCHS']+1):  # 에포크 설정
    model.train()
    running_loss=0.0

    for img,label in tqdm(iter(train_loader)):
      img,label=img.to(device),label.to(device)
      optimizer.zero_grad() # 배치마다 optimzier 초기화

      logit=model(img)
      loss=criterion(logit,label)

      loss.backward()
      optimizer.step() # 가중치 최적화
      running_loss+=loss.item()

    print('[%d] Train loss: %.10f' %(epoch, running_loss/len(train_loader)))

    if scheduler is not None:
      scheduler.step()

    # validation set 평가
    model.eval() # evaluatoin 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
    vali_loss=0.0
    correct=0

    with torch.no_grad(): # 파라미터 업데이트 안하기 때문에 no_grad 사용
      for img,label in tqdm(iter(vali_loader)):
        img,label=img.to(device),label.to(device)

        logit=model(img)
        vali_loss+= criterion(logit,label)
        pred=logit.argmax(dim=1,keepdim=True)  # 10개의 class 중 가장 값이 높은 것을 예측 label로 추출
        correct += pred.eq(label.view_as(pred)).sum().item()  # 예측값과 실제값이 맞으면 1 아니면 0으로 합산
    vali_acc=100 * correct/len(vali_loader.dataset)

    print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))
    
    if best_loss > vali_loss:
      best_loss=vali_loss
      torch.save(model.state_dict(),'/content/drive/MyDrive/data/save/best_model.pth')
      print('Model Saved')

In [ ]:
import torch.optim as optim # 최적화 알고리즘들이 포함힘

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = resnet.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

In [ ]:
train(resnet, optimizer, train_loader, scheduler, device)

In [ ]:
def predict(model,test_loader,device):
  model.eval()
  model_pred=[]

  with torch.no_grad():
    for img in tqdm(iter(test_loader)):
      img=img.to(device)

      pred_logit=model(img)
      pred_logit=pred_logit.argmax(dim=1,keepdim=True).squeeze(1)
      model_pred.extend(pred_logit.tolist())
  return model_pred

In [ ]:
test_dataset=CustomDataset(test_img_path,None,train_mode=False,transforms=test_transform)
test_loader=DataLoader(test_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=False,num_workers=0)

checkpoint=torch.load('/content/drive/MyDrive/data/save/best_model.pth')
Predictor=resnet.to(device)
Predictor.load_state_dict(checkpoint)

preds=predict(Predictor,test_loader,device)
preds[0:5]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
submission=pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv')
submission['label']=preds

In [ ]:
submission.to_csv('/content/drive/MyDrive/data/save/submission_resnet50_2.csv',index=False)